In [9]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

class TicTacToe:
    def __init__(self):
        self.board = np.array([['' for _ in range(3)] for _ in range(3)])
        self.current_player = 'X'  # Humano es 'X', IA es 'O'
        self.game_over = False
        self.info_label = widgets.Label(value="Tu turno (X)")
        display(self.info_label)
        self.create_board()

    def create_board(self):
        self.buttons = []
        for i in range(3):
            row = []
            for j in range(3):
                button = widgets.Button(description='', layout=widgets.Layout(width='80px', height='80px'))
                button.style.button_color = 'lightgray'
                button.on_click(self.on_button_click(i, j))
                row.append(button)
            self.buttons.append(row)
        self.board_ui = widgets.GridBox(
            children=[button for row in self.buttons for button in row],
            layout=widgets.Layout(
                width='270px',
                grid_template_columns='repeat(3, 90px)',
                grid_template_rows='repeat(3, 90px)'
            )
        )
        display(self.board_ui)

    def on_button_click(self, row, col):
        def handle_click(button):
            if self.board[row][col] == '' and not self.game_over and self.current_player == 'X':
                self.board[row][col] = 'X'
                button.description = 'X'
                button.style.button_color = 'lightblue'
                if self.check_winner():
                    self.end_game()
                else:
                    self.current_player = 'O'
                    self.info_label.value = "Turno de la IA (O)"
                    self.ai_move()
        return handle_click

    def ai_move(self):
        best_score = -float('inf')
        best_move = None
        temp_board = self.board.copy()
        for i in range(3):
            for j in range(3):
                if temp_board[i][j] == '':
                    temp_board[i][j] = 'O'
                    score = self.minimax(temp_board, 0, False, -float('inf'), float('inf'))
                    temp_board[i][j] = ''
                    if score > best_score:
                        best_score = score
                        best_move = (i, j)
        if best_move:
            i, j = best_move
            self.board[i][j] = 'O'
            self.buttons[i][j].description = 'O'
            self.buttons[i][j].style.button_color = 'salmon'
        if self.check_winner():
            self.end_game()
        else:
            self.current_player = 'X'
            self.info_label.value = "Tu turno (X)"

    def minimax(self, board, depth, is_maximizing, alpha, beta):
        winner = self.get_winner(board)
        if winner or self.is_board_full(board):
            return self.get_score(winner, depth)
        if is_maximizing:
            max_eval = -float('inf')
            for i in range(3):
                for j in range(3):
                    if board[i][j] == '':
                        board[i][j] = 'O'
                        eval = self.minimax(board, depth + 1, False, alpha, beta)
                        board[i][j] = ''
                        max_eval = max(max_eval, eval)
                        alpha = max(alpha, eval)
                        if beta <= alpha:
                            break
            return max_eval
        else:
            min_eval = float('inf')
            for i in range(3):
                for j in range(3):
                    if board[i][j] == '':
                        board[i][j] = 'X'
                        eval = self.minimax(board, depth + 1, True, alpha, beta)
                        board[i][j] = ''
                        min_eval = min(min_eval, eval)
                        beta = min(beta, eval)
                        if beta <= alpha:
                            break
            return min_eval

    def get_winner(self, board):
        # Comprobamos filas, columnas y diagonales
        for i in range(3):
            if board[i][0] == board[i][1] == board[i][2] != '':
                return board[i][0]
            if board[0][i] == board[1][i] == board[2][i] != '':
                return board[0][i]
        if board[0][0] == board[1][1] == board[2][2] != '':
            return board[0][0]
        if board[0][2] == board[1][1] == board[2][0] != '':
            return board[0][2]
        return None

    def is_board_full(self, board):
        return not np.any(board == '')

    def get_score(self, winner, depth):
        if winner == 'O':
            return 10 - depth
        elif winner == 'X':
            return depth - 10
        else:
            return 0

    def check_winner(self):
        winner = self.get_winner(self.board)
        if winner:
            self.info_label.value = f"¡{'Has ganado' if winner == 'X' else 'La IA ha ganado'}!"
            self.game_over = True
            return True
        elif self.is_board_full(self.board):
            self.info_label.value = "Es un empate."
            self.game_over = True
            return True
        return False

    def end_game(self):
        for row in self.buttons:
            for button in row:
                button.disabled = True
        reset_button = widgets.Button(description='Reiniciar Juego', layout=widgets.Layout(width='270px'))
        reset_button.on_click(self.reset_game)
        display(reset_button)

    def reset_game(self, button):
        clear_output()
        self.__init__()

# Ejecutamos el juego
game = TicTacToe()



Label(value='Tu turno (X)')

GridBox(children=(Button(layout=Layout(height='80px', width='80px'), style=ButtonStyle(button_color='lightgray…